<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/final_lstm_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
import csv
import pandas as pd 
    
# making dataframe 
df_human = pd.read_csv("new_human.csv") 
df_human.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_human)

df_ctrl = pd.read_csv("new_ctrl.csv") 
df_ctrl.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_ctrl)

df_gpt = pd.read_csv("new_gpt.csv") 
df_gpt.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_gpt)

df_gpt2 = pd.read_csv("new_gpt2.csv") 
df_gpt2.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_gpt2)

df_grover = pd.read_csv("new_grover.csv") 
df_grover.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_grover)

df_xlm = pd.read_csv("new_xlm.csv") 
df_xlm.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_xlm)

df_xlnet = pd.read_csv("new_xlnet.csv") 
df_xlnet.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_xlnet)

df_pplm = pd.read_csv("new_pplm.csv") 
df_pplm.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_pplm)

df_fair = pd.read_csv("new_fair.csv") 
df_fair.drop(columns=["Unnamed: 0"], inplace=True)
# print(df_fair)

df_gpt3 = pd.read_csv("final_text_curie_output.csv") 
df_gpt3.drop(columns=["Prompts"], inplace=True)
df_gpt3.drop(columns=["Unnamed: 0"], inplace=True)
df_gpt3 = df_gpt3.rename(columns={'Responses': 'Generation', 'Model Name': 'label'})
# print(df_gpt3)

df_instructgpt = pd.read_csv("final_instructgpt_results.csv") 
df_instructgpt.drop(columns=["Title"], inplace=True)
df_instructgpt.drop(columns=["Unnamed: 0"], inplace=True)
df_instructgpt = df_instructgpt.rename(columns={'InstructGPT_result': 'Generation', 'Model': 'label'})
# print(df_instructgpt)

result = pd.concat([df_ctrl, df_gpt, df_gpt2, df_grover,df_xlm, df_xlnet, df_pplm, df_fair, df_gpt3, df_instructgpt], axis=0)
result.drop(columns=["number"], inplace=True)
print(result)
df = result

                                             Generation        label
0     latest headlines on cnn business tl;dr the u.s...         ctrl
1     china wants to take a victory lap over its han...         ctrl
2     coronavirus disinformation creates challenges ...         ctrl
3     china coronavirus: eating wild animals made il...         ctrl
4     china's economy could shrink for the first tim...         ctrl
...                                                 ...          ...
1061         100%. All parts of your body are your own.  InstructGPT
1062  1. Establish regular cleaning schedules for al...  InstructGPT
1063  In some cities around the world, including Tok...  InstructGPT
1064  . The BBC News app is a comprehensive and easy...  InstructGPT
1065  The BBC is implementing several steps to stren...  InstructGPT

[10661 rows x 2 columns]


In [13]:
import pandas as pd

# convert dataframe to csv file
result.to_csv('final_task3_data.csv', index=False)  # specify index=False to exclude index column in output file


In [14]:
import pandas as pd

df = pd.read_csv('final_task3_data.csv')

print(df.head())


                                          Generation label
0  latest headlines on cnn business tl;dr the u.s...  ctrl
1  china wants to take a victory lap over its han...  ctrl
2  coronavirus disinformation creates challenges ...  ctrl
3  china coronavirus: eating wild animals made il...  ctrl
4  china's economy could shrink for the first tim...  ctrl


In [15]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [18]:
import pandas as pd

In [17]:
df[['Generation']] = df[['Generation']]. astype(str)

In [ ]:
df = df[['Generation','label']]

In [20]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',  include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

pt = [TEXT.preprocess(x) for x in df['text']]

TEXT.build_vocab(
    pt, max_size = 25000,
    vectors='glove.6B.50d', unk_init = torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399999/400000 [00:11<00:00, 33442.45it/s]


In [21]:
label_to_index= {label: index for index, label in enumerate(df['label'].unique())}


In [22]:
df['label_new'] = df['label'].apply(lambda y: label_to_index[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], fields)

In [24]:
df_new=df[['text','label_new']]


In [25]:
examples=[]
for i,row in df_new.iterrows():
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [26]:
itrr=0
for i in examples:
  if type(i.label) != int:
    itrr+=1

In [28]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [30]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [34]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [36]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,720,611 trainable parameters


In [39]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [40]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])

In [41]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])


In [42]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [43]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [44]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [45]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [47]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [50]:
N_EPOCHS = 15
path = F"/content/gdrive/My Drive/bilstm_model_task3.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 0.548 | Train Acc: 78.89%
	 Val. Loss: 0.587 |  Val. Acc: 76.50%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.479 | Train Acc: 79.87%
	 Val. Loss: 0.931 |  Val. Acc: 70.44%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.466 | Train Acc: 81.06%
	 Val. Loss: 0.583 |  Val. Acc: 76.69%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.491 | Train Acc: 81.18%
	 Val. Loss: 0.491 |  Val. Acc: 79.88%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss: 0.391 | Train Acc: 83.90%
	 Val. Loss: 0.540 |  Val. Acc: 77.12%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.375 | Train Acc: 84.25%
	 Val. Loss: 0.492 |  Val. Acc: 80.38%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.329 | Train Acc: 86.25%
	 Val. Loss: 0.450 |  Val. Acc: 83.00%
Epoch: 08 | Epoch Time: 0m 4s
	Train Loss: 0.308 | Train Acc: 87.22%
	 Val. Loss: 0.567 |  Val. Acc: 77.69%
Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.287 | Train Acc: 88.33%
	 Val. Loss: 0.458 |  Val. Acc: 83.38%
Epoch: 10 | Epoch Time: 0m 3

In [51]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.386 | Test Acc: 86.14%


In [52]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# load the trained model
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [53]:
y_true = []
y_pred = []

model.load_state_dict(torch.load("/content/gdrive/My Drive/bilstm_model_task3.pt"))
model=model.to(device)

for batch in test_iter:
    text, labels = batch
    text,text_length=text
    predictions = model(text,text_length).squeeze(1)
    y_true.extend(labels.tolist())
    y_pred.extend(predictions.argmax(dim=1).tolist())  # convert to list of labels

# calculate the evaluation metrics using sklearn's functions
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)
import pandas as pd


Accuracy: 0.8408450704225352
Precision: 0.8503655115924165
Recall: 0.8408450704225352
F1 score: 0.8390460803097304
